In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, initcap, col
from pyspark.sql.types import StringType

In [2]:
spark = SparkSession.builder.appName('UDF').getOrCreate()

22/04/04 12:47:21 WARN Utils: Your hostname, BTCCHL0018 resolves to a loopback address: 127.0.1.1; using 192.168.0.233 instead (on interface wlp44s0)
22/04/04 12:47:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/04 12:47:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
column_names = ["id", "name"]
data = [("1", "abc def"),
       ("2", "ghi jkl"),
       ("3", "mno pqr"),
       ("4", "stu vwx")]

In [4]:
df = spark.createDataFrame(data = data, schema = column_names)
df.show()

+---+-------+
| id|   name|
+---+-------+
|  1|abc def|
|  2|ghi jkl|
|  3|mno pqr|
|  4|stu vwx|
+---+-------+



In [5]:
def convertCase(str):
    retStr = ""
    arr = str.split(" ")
    for i in arr:
        retStr = retStr + i[0].upper() + i[1:] + " "
    return retStr

In [6]:
convertUDF = udf(lambda z: convertCase(z), StringType())
convertUDF

<function __main__.<lambda>(z)>

In [7]:
df.select("id", convertUDF("name").alias("Name")).show(truncate=False)

+---+--------+
|id |Name    |
+---+--------+
|1  |Abc Def |
|2  |Ghi Jkl |
|3  |Mno Pqr |
|4  |Stu Vwx |
+---+--------+



In [8]:
spark.udf.register("convertUDF", convertCase, StringType())
df.createOrReplaceTempView("Name_TABLE")
spark.sql('select id, convertUDF(name) as Name from Name_TABLE')

DataFrame[id: string, Name: string]

In [9]:
#Predefined for this UDF is initcap
df = df.select("id", initcap("name").alias("Name"))

In [10]:
def checkIfUpper(str):
    li = []
    arr = str.split(" ")
    for i in arr:
        li.append(i.isupper())
    return all(li)

In [11]:
checkUDF = udf(lambda z: checkIfUpper(z), StringType())
checkUDF

<function __main__.<lambda>(z)>

In [12]:
df.show()

+---+-------+
| id|   Name|
+---+-------+
|  1|Abc Def|
|  2|Ghi Jkl|
|  3|Mno Pqr|
|  4|Stu Vwx|
+---+-------+



In [13]:
@udf(returnType=StringType())
def convertToUpper(str):
    return str.upper()

In [14]:
df = df.withColumn("Upper-Case Name", convertToUpper(col("Name")))
df.show()

+---+-------+---------------+
| id|   Name|Upper-Case Name|
+---+-------+---------------+
|  1|Abc Def|        ABC DEF|
|  2|Ghi Jkl|        GHI JKL|
|  3|Mno Pqr|        MNO PQR|
|  4|Stu Vwx|        STU VWX|
+---+-------+---------------+



In [17]:
df = df.withColumn("Is Upper name", checkUDF(col("Upper-Case Name")))

In [18]:
df.show(truncate= False)

+---+-------+---------------+-------------+
|id |Name   |Upper-Case Name|Is Upper name|
+---+-------+---------------+-------------+
|1  |Abc Def|ABC DEF        |true         |
|2  |Ghi Jkl|GHI JKL        |true         |
|3  |Mno Pqr|MNO PQR        |true         |
|4  |Stu Vwx|STU VWX        |true         |
+---+-------+---------------+-------------+



In [21]:
spark.sql("select id, convertUDF(name) as Name from Name_TABLE where Name is not null and convertUDF(name) like '%Abc%'").show()

+---+--------+
| id|    Name|
+---+--------+
|  1|Abc Def |
+---+--------+



In [24]:
columns = ["Seqno","Name"]
data = [("1", "john jones"),
    ("2", "tracey smith"),
    ("3", "amy sanders"),
    ('4',None)]

df2 = spark.createDataFrame(data=data,schema=columns)
df2.show(truncate=False)
df2.createOrReplaceTempView("NAME_TABLE2")



+-----+------------+
|Seqno|Name        |
+-----+------------+
|1    |john jones  |
|2    |tracey smith|
|3    |amy sanders |
|4    |null        |
+-----+------------+



In [27]:
# Throws error since the df has empty values and split could not be used on it
spark.sql("select convertUDF(Name) from NAME_TABLE2") \
     .show(truncate=False) 

22/04/04 13:01:08 ERROR Executor: Exception in task 2.0 in stage 41.0 (TID 111)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/tmp/ipykernel_33750/2304883466.py", line 3, in convertCase
AttributeError: 'NoneType' object has no attribute 'split'

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:555)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:86)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:68)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:508)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:491)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.cat

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/tmp/ipykernel_33750/2304883466.py", line 3, in convertCase
AttributeError: 'NoneType' object has no attribute 'split'


In [28]:

spark.udf.register("_nullsafeUDF", lambda str: convertCase(str) if not str is None else "" , StringType())

spark.sql("select _nullsafeUDF(Name) from NAME_TABLE2") \
     .show(truncate=False)

spark.sql("select Seqno, _nullsafeUDF(Name) as Name from NAME_TABLE2 " + \
          " where Name is not null and _nullsafeUDF(Name) like '%John%'") \
     .show(truncate=False)    


22/04/04 13:01:08 WARN SimpleFunctionRegistry: The function _nullsafeudf replaced a previously registered function.


+------------------+
|_nullsafeUDF(Name)|
+------------------+
|John Jones        |
|Tracey Smith      |
|Amy Sanders       |
|                  |
+------------------+

+-----+-----------+
|Seqno|Name       |
+-----+-----------+
|1    |John Jones |
+-----+-----------+

